In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# A. gen mv cost for assets
# B. aggr to port
# C. stack assets and port

##############################
# A. gen mv cost for assets
##############################

#------------------------------
#-- 0. Read in Assets, Strats
#------------------------------
assets_df = pd.read_csv('D1_assets.csv')
strats_df = pd.read_csv('S02 M1 9 Stocks.csv')

#-------------------------------------
#-- 1. Merge assets pricd onto strats
#-------------------------------------
# day 1 buy
PRICE_COL = '"Adj Close"' # SQL needs double quotes on field name w space

q = """SELECT 
      drv.PID
     ,drv.AID
     ,drv.Date
     ,drv.qty_chg
     ,a.{price_col}
    FROM strats_df drv
    LEFT JOIN assets_df a on a.AID=drv.AID AND a.Date=drv.Date
    ORDER BY drv.PID, drv.AID, drv.Date
    ;""".format(price_col = PRICE_COL)

df = pysqldf(q)

#----------------------------------------
#-- 2. For each PID AID pair, gen mv cost
#----------------------------------------
## market value
df['cum_qty'] = df.groupby(['PID','AID']).cumsum()['qty_chg']
df['mv'] = df['cum_qty'] * df['Adj Close']

## cost
df['cost_chg'] = df['qty_chg']*df['Adj Close']
df['cost'] = df.groupby(['PID','AID']).cumsum()['cost_chg']



##############################
# B. gen mv cost for assets
##############################
q = """SELECT 
      PID, Date
     ,sum({mv_col}) as {mv_col}
     ,sum({cost_col}) as {cost_col}
    FROM 
      df
    GROUP BY PID, Date
    ORDER BY PID, Date
    ;""".format(mv_col='mv',cost_col='cost')
p_df = pysqldf(q)


##############################
# C. stack assets and port
##############################
#--------------------------------------
#-- 1. Union
#--------------------------------------
p_df['AID'] = p_df['PID']

q = """SELECT {cols} FROM df
       UNION
       SELECT {cols} FROM p_df
    ;""".format(cols='PID, AID, Date, mv, cost')
df = pysqldf(q)
# ref code: pd.concat([df1, df2])

#------------------------------
#-- 3. Output to csv
#------------------------------
# Key: PID AID Date
df.to_csv('M02 M1 9 Stocks.csv',index=False)

In [3]:
df.head()

,PID,AID,Date,mv,cost
0,S02,AAPL,2014-03-03,0.0,0.0
1,S02,AAPL,2014-03-04,0.0,0.0
2,S02,AAPL,2014-03-05,0.0,0.0
3,S02,AAPL,2014-03-06,0.0,0.0
4,S02,AAPL,2014-03-07,0.0,0.0
